In [2]:
import torch
import torch.nn as nn
from torchdiffeq import odeint
import xarray as xr
from torch.utils.data import DataLoader, TensorDataset

In [3]:
dataset_path = 'gs://weatherbench2/datasets/era5/1959-2022-6h-1440x721.zarr'
ds = xr.open_zarr(dataset_path)
print(ds)

<xarray.Dataset> Size: 41TB
Dimensions:                                           (time: 92044,
                                                       latitude: 721,
                                                       longitude: 1440,
                                                       level: 13)
Coordinates:
  * latitude                                          (latitude) float32 3kB ...
  * level                                             (level) int64 104B 50 ....
  * longitude                                         (longitude) float32 6kB ...
  * time                                              (time) datetime64[ns] 736kB ...
Data variables: (12/38)
    10m_u_component_of_wind                           (time, latitude, longitude) float32 382GB ...
    10m_v_component_of_wind                           (time, latitude, longitude) float32 382GB ...
    10m_wind_speed                                    (time, latitude, longitude) float32 382GB ...
    2m_temperature           

In [4]:
print("\nVariables disponibles:")
print(list(ds.data_vars))


Variables disponibles:
['10m_u_component_of_wind', '10m_v_component_of_wind', '10m_wind_speed', '2m_temperature', 'angle_of_sub_gridscale_orography', 'anisotropy_of_sub_gridscale_orography', 'geopotential', 'geopotential_at_surface', 'high_vegetation_cover', 'lake_cover', 'lake_depth', 'land_sea_mask', 'low_vegetation_cover', 'mean_sea_level_pressure', 'sea_ice_cover', 'sea_surface_temperature', 'slope_of_sub_gridscale_orography', 'soil_type', 'specific_humidity', 'standard_deviation_of_filtered_subgrid_orography', 'standard_deviation_of_orography', 'surface_pressure', 'temperature', 'toa_incident_solar_radiation', 'toa_incident_solar_radiation_12hr', 'toa_incident_solar_radiation_24hr', 'toa_incident_solar_radiation_6hr', 'total_cloud_cover', 'total_column_water_vapour', 'total_precipitation_12hr', 'total_precipitation_24hr', 'total_precipitation_6hr', 'type_of_high_vegetation', 'type_of_low_vegetation', 'u_component_of_wind', 'v_component_of_wind', 'vertical_velocity', 'wind_speed']

In [5]:
variables = [
    '10m_u_component_of_wind',
    '10m_v_component_of_wind',
    '2m_temperature',
    'geopotential',
    'specific_humidity'
]
subset = ds[variables].sel(
    time=slice('2010-01-01', '2010-01-02'),
    latitude=slice(-17.0, -56.0),          
    longitude=slice(280.0, 310.0) 
)

print(subset)

<xarray.Dataset> Size: 18MB
Dimensions:                  (time: 8, latitude: 157, longitude: 121, level: 13)
Coordinates:
  * latitude                 (latitude) float32 628B -17.0 -17.25 ... -56.0
  * level                    (level) int64 104B 50 100 150 200 ... 850 925 1000
  * longitude                (longitude) float32 484B 280.0 280.2 ... 310.0
  * time                     (time) datetime64[ns] 64B 2010-01-01 ... 2010-01...
Data variables:
    10m_u_component_of_wind  (time, latitude, longitude) float32 608kB ...
    10m_v_component_of_wind  (time, latitude, longitude) float32 608kB ...
    2m_temperature           (time, latitude, longitude) float32 608kB ...
    geopotential             (time, level, latitude, longitude) float32 8MB ...
    specific_humidity        (time, level, latitude, longitude) float32 8MB ...


In [6]:
def preprocess_dataset(subset, variables):
    normalized_vars = {}
    for var in variables:
        data = subset[var].values  # Extraer datos
        mean = data.mean()
        std = data.std()
        normalized_data = (data - mean) / std
        normalized_vars[var] = normalized_data

    tensors = [torch.tensor(normalized_vars[var], dtype=torch.float32) for var in variables]
    tensor_data = torch.stack(tensors, dim=1) 
    return tensor_data

In [ ]:
for var in variables:
    print(f"Variable: {var}, Shape: {subset[var].shape}")

Variable: 10m_u_component_of_wind, Shape: (8, 157, 121)
Variable: 10m_v_component_of_wind, Shape: (8, 157, 121)
Variable: 2m_temperature, Shape: (8, 157, 121)
Variable: geopotential, Shape: (8, 13, 157, 121)
Variable: specific_humidity, Shape: (8, 13, 157, 121)


In [8]:
subset['geopotential'] = subset['geopotential'].sel(level=500)


In [9]:
subset['specific_humidity'] = subset['specific_humidity'].sel(level=subset['specific_humidity']['level'].values[0])

In [10]:
data_tensor = preprocess_dataset(subset, variables)


In [11]:
# División
time_steps = data_tensor.shape[0]
train_split = int(0.7 * time_steps)
val_split = int(0.15 * time_steps)

train_data = data_tensor[:train_split]
val_data = data_tensor[train_split:train_split + val_split]
test_data = data_tensor[train_split + val_split:]

print(f"Entrenamiento: {train_data.shape}, Validación: {val_data.shape}, Prueba: {test_data.shape}")

Entrenamiento: torch.Size([5, 5, 157, 121]), Validación: torch.Size([1, 5, 157, 121]), Prueba: torch.Size([2, 5, 157, 121])


In [12]:
print(f"Tamaño del conjunto de entrenamiento: {train_data.shape[0]}")
print(f"Tamaño del conjunto de validación: {val_data.shape[0]}")
print(f"Tamaño del conjunto de prueba: {test_data.shape[0]}")


Tamaño del conjunto de entrenamiento: 5
Tamaño del conjunto de validación: 1
Tamaño del conjunto de prueba: 2


In [13]:
window_size_train = min(5, train_data.shape[0])
window_size_val = min(5, val_data.shape[0])
window_size_test = min(5, test_data.shape[0])

print(f"Ventanas ajustadas - Entrenamiento: {window_size_train}, Validación: {window_size_val}, Prueba: {window_size_test}")

Ventanas ajustadas - Entrenamiento: 5, Validación: 1, Prueba: 2


In [14]:
def create_temporal_windows(data, window_size, stride):
  """
  Crea ventanas temporales de tamaño window_size con un stride específico.
  """
  if data.shape[0] < window_size:
      print(f"No se pueden crear ventanas: tamaño insuficiente de datos ({data.shape[0]} < {window_size}).")
      return torch.empty(0)  # Retorna un tensor vacío si no hay suficientes datos

  windows = []
  for i in range(0, data.shape[0] - window_size + 1, stride):
      windows.append(data[i:i + window_size])
  return torch.stack(windows)


In [15]:
train_windows = create_temporal_windows(train_data, window_size=5, stride=1)
val_windows = create_temporal_windows(val_data, window_size=1, stride=1)
test_windows = create_temporal_windows(test_data, window_size=2, stride=1)
print(f"Ventanas de entrenamiento: {train_windows.shape}")
print(f"Ventanas de validación: {val_windows.shape}")
print(f"Ventanas de prueba: {test_windows.shape}")


Ventanas de entrenamiento: torch.Size([1, 5, 5, 157, 121])
Ventanas de validación: torch.Size([1, 1, 5, 157, 121])
Ventanas de prueba: torch.Size([1, 2, 5, 157, 121])


In [16]:
def duplicate_windows(windows, factor=3):
    """
    Duplica las ventanas existentes para crear un conjunto más grande.
    """
    return torch.cat([windows for _ in range(factor)], dim=0)

if train_windows.shape[0] > 0:
    train_windows = duplicate_windows(train_windows, factor=3)

if val_windows.shape[0] > 0:
    val_windows = duplicate_windows(val_windows, factor=3)

if test_windows.shape[0] > 0:
    test_windows = duplicate_windows(test_windows, factor=3)

print(f"Nuevas ventanas de entrenamiento: {train_windows.shape}")
print(f"Nuevas ventanas de validación: {val_windows.shape}")
print(f"Nuevas ventanas de prueba: {test_windows.shape}")


Nuevas ventanas de entrenamiento: torch.Size([3, 5, 5, 157, 121])
Nuevas ventanas de validación: torch.Size([3, 1, 5, 157, 121])
Nuevas ventanas de prueba: torch.Size([3, 2, 5, 157, 121])


In [17]:
train_loader = None
val_loader = None
test_loader = None

if train_windows.shape[0] > 1:
    train_dataset = TensorDataset(train_windows[:-1], train_windows[1:])
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
else:
    print("Conjunto de entrenamiento insuficiente para crear DataLoader.")

if val_windows.shape[0] > 1:
    val_dataset = TensorDataset(val_windows[:-1], val_windows[1:])
    val_loader = DataLoader(val_dataset, batch_size=1)
else:
    print("Conjunto de validación insuficiente para crear DataLoader.")

if test_windows.shape[0] > 1:
    test_dataset = TensorDataset(test_windows[:-1], test_windows[1:])
    test_loader = DataLoader(test_dataset, batch_size=1)
else:
    print("Conjunto de prueba insuficiente para crear DataLoader.")

# Confirmar los DataLoaders creados
if train_loader:
    print(f"Lotes de entrenamiento: {len(train_loader)}")
if val_loader:
    print(f"Lotes de validación: {len(val_loader)}")
if test_loader:
    print(f"Lotes de prueba: {len(test_loader)}")

Lotes de entrenamiento: 2
Lotes de validación: 2
Lotes de prueba: 2


In [18]:
# Bloque Residual Simplificado
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()

        # Proyección para igualar dimensiones si in_channels != out_channels
        self.projection = nn.Conv2d(in_channels, out_channels, kernel_size=1) if in_channels != out_channels else None

    def forward(self, x):
        residual = x
        if self.projection:
            residual = self.projection(x)  # Proyectar para igualar dimensiones

        x = self.relu(self.conv1(x))
        x = self.conv2(x)
        return self.relu(x + residual)


# Modelo CNN Simplificado
class ClimateResNet2D(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.resblock1 = ResidualBlock(in_channels, hidden_channels)
        self.resblock2 = ResidualBlock(hidden_channels, hidden_channels)
        self.output_layer = nn.Conv2d(hidden_channels, in_channels, kernel_size=3, padding=1)

    def forward(self, x):
        x = self.resblock1(x)
        x = self.resblock2(x)
        return self.output_layer(x)

# Restricciones PDE Simplificadas
def pde_loss(pred, vx, vy):
    """
    Calcula la pérdida basada en la ecuación de advección.
    """
    pred.requires_grad_(True)
    u_t = torch.gradient(pred, dim=0)[0]  # Gradiente respecto al tiempo
    u_x = torch.gradient(pred, dim=2)[0]  # Gradiente respecto a x
    u_y = torch.gradient(pred, dim=3)[0]  # Gradiente respecto a y

    advection = u_t + vx * u_x + vy * u_y
    return torch.mean(advection ** 2)

class ODEBlock(nn.Module):
    def __init__(self, func, features):
        super().__init__()
        self.func = func
        self.features = features  # Número de características planas (channels * height * width)

    def forward(self, x, t):
        # Aplanar dimensiones espaciales
        batch_size, channels, height, width = x.shape
        x = x.view(batch_size, -1)  # Aplanar a (batch_size, features)

        # Pasar por el solver ODE
        x = odeint(self.func, x, t, method='rk4')

        # Restaurar dimensiones espaciales
        x = x.view(batch_size, channels, height, width)
        return x

class ODEFunc(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, hidden_dim)
        )

    def forward(self, t, x):
        return self.net(x)

# Modelo Final Simplificado
class ClimateModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, height, width):
        super().__init__()
        self.cnn = ClimateResNet2D(in_channels, hidden_channels)
        
        # Calcular el número de características planas para el ODEBlock
        features = hidden_channels * height * width
        
        # Inicializar ODEBlock con el número correcto de características
        self.ode_block = ODEBlock(ODEFunc(features), features)

    def forward(self, x, t):
        x = self.cnn(x)  # CNN para efectos locales
        x = self.ode_block(x, t)  # ODE para continuidad temporal
        return x

In [19]:
def train_model(model, train_loader, val_loader, epochs, optimizer, device):
    model.train()
    for epoch in range(epochs):
        total_loss = 0.0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            # Aplanar las dimensiones (batch_size, window_size * channels, height, width)
            batch_size, window_size, channels, height, width = inputs.shape
            inputs = inputs.view(batch_size, window_size * channels, height, width)
            targets = targets.view(batch_size, window_size * channels, height, width)

            t = torch.linspace(0, 1, steps=inputs.size(1)).to(device)  # Normalizar tiempos

            # Forward
            outputs = model(inputs, t)

            # Pérdida
            loss = nn.functional.mse_loss(outputs, targets)

            # Backpropagación
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")


In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
features = 64 * 157 * 121  # channels * height * width
height, width = 157, 121  # Dimensiones espaciales
in_channels = 25  # Combina window_size * variables climáticas
hidden_channels = 64  # Número de canales ocultos después de la CNN
model = ClimateModel(in_channels=in_channels, hidden_channels=hidden_channels, height=height, width=width).to(device)
model.ode_block = ODEBlock(ODEFunc(features), features).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 5
train_model(model, train_loader, val_loader, epochs, optimizer, device)

: 